In [1]:
import tensorflow as tf
import joblib, pathlib
import numpy as np


# Añadir src/ al path para poder importar config
PROJECT_ROOT = pathlib.Path().resolve().parent.parent  
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
from src import config as cfg

# --- Cargar dataset procesado ---
ruta = cfg.DATA / "processed" / "gru5d_data.pkl"
print("📦 Cargando datos desde:", ruta)

data = joblib.load(ruta)
X, y = data["X"], data["y"]

print(f"✅ X shape: {X.shape}")     # (n_muestras, 60, 2N)
print(f"✅ y shape: {y.shape}")     # (n_muestras, N activos)
print(f"🧾 Nº activos: {y.shape[1]}")

📦 Cargando datos desde: C:\Users\ferra\Documents\TFM\data\processed\gru5d_data.pkl
✅ X shape: (4450, 60, 80)
✅ y shape: (4450, 40)
🧾 Nº activos: 40


In [3]:
# --- Crear modelo GRU ---
print("🛠️  Definiendo arquitectura...")

inputs = tf.keras.Input(shape=X.shape[1:])
x = tf.keras.layers.GRU(64, return_sequences=True)(inputs)
x = tf.keras.layers.GRU(32)(x)
outputs = tf.keras.layers.Dense(y.shape[1])(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()


🛠️  Definiendo arquitectura...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 60, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 60, 64)         │        28,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │         1,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,760 (151.41 KB)

 Trainable params: 38,760 (151.41 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# --- Entrenamiento ---
ckpt_path = pathlib.Path(cfg.MODELS) / "gru5d.keras"
print("📁 Checkpoint se guardará en:", ckpt_path)


print("🔍 NaNs en X:", np.isnan(X).sum())
print("🔍 NaNs en y:", np.isnan(y).sum())
print("🔍 Inf en X :", np.isinf(X).sum())
print("🔍 Inf en y :", np.isinf(y).sum())
print("🔍 Rango X  :", np.min(X), "→", np.max(X))
print("🔍 Rango y  :", np.min(y), "→", np.max(y))

es_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X, y,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[es_cb],
    verbose=2
)


print("🏁 Entrenamiento finalizado.")

📁 Checkpoint se guardará en: C:\Users\ferra\Documents\TFM\models\gru5d.keras
🔍 NaNs en X: 0
🔍 NaNs en y: 0
🔍 Inf en X : 0
🔍 Inf en y : 0
🔍 Rango X  : -6.96164015604789 → 7.819829095741229
🔍 Rango y  : -0.1500119647442526 → 1.225082801981301
Epoch 1/50
112/112 - 10s - 88ms/step - loss: 0.0041 - root_mean_squared_error: 0.0637 - val_loss: 7.7769e-04 - val_root_mean_squared_error: 0.0279
Epoch 2/50
112/112 - 7s - 65ms/step - loss: 4.7759e-04 - root_mean_squared_error: 0.0219 - val_loss: 3.0241e-04 - val_root_mean_squared_error: 0.0174
Epoch 3/50
112/112 - 7s - 63ms/step - loss: 2.8567e-04 - root_mean_squared_error: 0.0169 - val_loss: 1.9279e-04 - val_root_mean_squared_error: 0.0139
Epoch 4/50
112/112 - 7s - 64ms/step - loss: 2.2513e-04 - root_mean_squared_error: 0.0150 - val_loss: 1.5032e-04 - val_root_mean_squared_error: 0.0123
Epoch 5/50
112/112 - 7s - 61ms/step - loss: 1.9966e-04 - root_mean_squared_error: 0.0141 - val_loss: 1.2820e-04 - val_root_mean_squared_error: 0.0113
Epoch 6/50
1